# Run benchmark

Loads the dataset and executes the bus factor analysis for each repo 10 times and collects the runtime metrics.

In [11]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

TIME_COLUMN = "Execution time series"

def read_dataset(filename):
    df = pd.read_csv(filename, index_col=0)
    for column in ["Stars", "Size", "Mentionable Users", "Commits"]:
        df[column] = df[column].astype('int')

    df = df.drop_duplicates(subset=['Repository'])
    df = df.query('Commits >= 100 & Language != None & Commits <= 20000')
    # Repos less or equal 10Gb size
    df = df.query('Size <= 10000000')
    
    # Add columns
    df.loc[:, TIME_COLUMN] = 0
    df[TIME_COLUMN] = df[TIME_COLUMN].astype('object')
    df.loc[:, "Bus Factor"] = 0
    df["Bus Factor"] = df["Bus Factor"].astype('int')
    return df

df = read_dataset("data/dataset.csv")
df.describe()

,Stars,Size,Mentionable Users,Commits,Bus Factor
count,943.000000,9.430000e+02,943.000000,943.000000,943.0
mean,29453.748674,2.209381e+05,513.545069,1638.586426,0.0
std,26277.948371,5.181760e+05,885.477256,2396.988686,0.0
min,8925.000000,5.170000e+02,1.000000,100.000000,0.0
25%,16191.500000,1.898350e+04,126.000000,279.500000,0.0
50%,21468.000000,6.143600e+04,255.000000,737.000000,0.0
75%,32708.500000,1.981075e+05,536.500000,1963.000000,0.0
max,367200.000000,6.060546e+06,10000.000000,19165.000000,0.0


In [12]:
# Merge datasets if needed
def merge(df, filename):
    prev_test = pd.read_csv(filename, index_col=0)
    for index, row in prev_test.iterrows():
        if row["Bus Factor"] != 0:
            idx = df.index[df['Repository'] == row["Repository"]]
            df.loc[idx, TIME_COLUMN] = row[TIME_COLUMN]
            df.loc[idx, 'Bus Factor'] = row["Bus Factor"]
    return df

# df = merge(df, "data/result.csv")
df.query('Repository == "flutter/flutter"')

,Repository,Clone URL,Stars,Size,Mentionable Users,Commits,Language,Bus Factor,Execution time series
5,flutter/flutter,https://github.com/flutter/flutter.git,153561,259459,1277,8307,Dart,0,0.0


In [13]:
df.sort_values(by=["Commits"], inplace=True)
df.describe()

,Stars,Size,Mentionable Users,Commits,Bus Factor
count,943.000000,9.430000e+02,943.000000,943.000000,943.0
mean,29453.748674,2.209381e+05,513.545069,1638.586426,0.0
std,26277.948371,5.181760e+05,885.477256,2396.988686,0.0
min,8925.000000,5.170000e+02,1.000000,100.000000,0.0
25%,16191.500000,1.898350e+04,126.000000,279.500000,0.0
50%,21468.000000,6.143600e+04,255.000000,737.000000,0.0
75%,32708.500000,1.981075e+05,536.500000,1963.000000,0.0
max,367200.000000,6.060546e+06,10000.000000,19165.000000,0.0


In [ ]:
import pandas as pd
import statistics
import requests
import time
import json

API_BASE_PATH = "http://localhost:8080"
ARTIFACTS_BASE_PATH = "../compose-workdir/artifacts/"

cnt = 0
failed = []


for index, row in df.iterrows():
    cnt += 1
    task_is_failed = False
    elapsedMillis = []
    busFactor = None

    print("=" * 12)
    print(f"[{cnt}/{len(df)}] " + row["Repository"])
    print()
    if row["Repository"] in failed:
        print("Not supported")
        continue
    if row["Bus Factor"] != 0:
        print("Already calculated")
        continue

    for i in range(10):
        payload = {
            "cloneUrl": row["Clone URL"],
            "owner": row["Repository"].split("/")[0],
            "repo": row["Repository"].split("/")[1],
        }
        r = requests.post(API_BASE_PATH + "/api/task/submit", json=payload)
        print(r.text + ": " + row["Repository"])

        task_is_running = True
        while task_is_running:
            time.sleep(0.5)
            status = requests.get(API_BASE_PATH + "/api/task/events")
            for msg in status.json():
                print(msg)
                if msg["status"] == "FAILED":
                    task_is_running = False
                    task_is_failed = True
                if msg["status"] == "DONE":
                    task_is_running = False

        if task_is_failed:
            failed.append(row["Repository"])
            df.drop(index, inplace=True)
            break

        path = ARTIFACTS_BASE_PATH + row["Repository"] + "/metrics.json"
        metrics = None
        with open(path, "r") as file:
            metrics = json.load(file)
        busFactor = int(metrics["busFactor"])
        elapsedMillis.append(int(metrics["elapsedMillis"]))
    if task_is_failed:
        continue

    df.at[index, TIME_COLUMN] = elapsedMillis
    df.at[index, "Bus Factor"] = busFactor


In [10]:
import datetime

now = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
with open("data/result_" + now + ".csv", "w+") as output:
    output.write(df.to_csv())


In [ ]:
print(len(failed), len(df), failed)